In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta

import sys

from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

Import Datasets

In [2]:
rating_df = pd.read_csv("../data/custom/final_ratings.csv")
games_df = pd.read_csv("../data/custom/game_learn_df.csv")
games_df.rename(columns={'BGGId':'bgg_id'},inplace=True)
rating_df.rename(columns={'BGGId':'bgg_id'},inplace=True)

In [3]:
rating_df

,bgg_id,Username,Rating
0,13,beastvol,8.0
1,118,beastvol,7.0
2,278,beastvol,7.0
3,823,beastvol,6.0
4,3076,beastvol,9.0
...,...,...,...
32307585,303123,LovingUniverse,6.0
32307586,303123,ecoboardgeek123,5.1
32307587,303123,xanthie,5.0
32307588,303123,Ilse23,4.0


### Predefined Prep Functions

In [4]:
def UserFrame(rating, games, UName):
    u_rating = rating[rating['Username'] == UName]
    combined_df = u_rating.merge(games, on='bgg_id')
    return(combined_df)

In [5]:
def minbo_reduction(minbo, wf, game_id):
    filtered_df = wf[wf['bgg_id'] == game_id]
    visitors = filtered_df['Username'].unique()
    filtered_df = wf[wf['Username'].isin(visitors)]
    count_table = filtered_df['bgg_id'].value_counts().reset_index()
    count_table.columns = ['bgg_id', 'count']
    # print(count_table['count'].describe())
    if ((count_table['count'].describe().iloc[6]) < minbo):
        print(f'minbo to large. new mibo set to {count_table["count"].describe().iloc[6]}')
        minbo = (count_table["count"].describe().iloc[6])
    count_table=count_table.loc[count_table['count']>=minbo,'bgg_id']
    return(count_table)

In [6]:
keep_users = set(
    (
        (rating_df.groupby(
            'Username'
        )[['bgg_id']].count()
        ).loc[(
            rating_df.groupby(
                'Username'
            )[['bgg_id']].count()
        )['bgg_id']>12].reset_index()
    )['Username']
)

In [7]:
redu_rating = rating_df.loc[rating_df['Username'].isin(keep_users)]

In [8]:
red_game = games_df.drop(columns=(['min_age', 'min_time']))

#Remove Multiple Wargames
drop_war = list(red_game.filter(regex='War').columns)
drop_war.remove('War Game')
red_game.drop(columns=(drop_war), inplace=True)

#Remove multiple Worker Placement
drop_worker = list(red_game.filter(regex='Worker').columns)
drop_worker.remove('Worker Placement')
red_game.drop(columns=(drop_worker), inplace=True)

#Remove multiple Dice
drop_dice = list(red_game.filter(regex='Dice').columns)
drop_dice.remove('Dice')
red_game.drop(columns=(drop_dice), inplace=True)

#Remove multiple Card
drop_card = list(red_game.filter(regex='Card').columns)
drop_card.remove('Card Game')
red_game.drop(columns=(drop_card), inplace=True)

#Remove multiple Animal
drop_animal = list(red_game.filter(regex='Animal').columns)
drop_animal.remove('Animals')
red_game.drop(columns=(drop_animal), inplace=True)


# red_game.columns[75:]
# list(red_game.filter(regex='Village').columns)

#### Scaling Columns

In [9]:
red_game.set_index('bgg_id',inplace=True)
# red_game.head(1)

In [10]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

scaler = MinMaxScaler()
for col in red_game.columns:
    colly = red_game.loc[:, col].values.reshape(-1, 1)
    red_game.loc[:, col] = scaler.fit_transform(colly)

# Print the scaled DataFrame
print("Scaled Dataset Using MinMaxScaler")
red_game['avg_rating'] = red_game['avg_rating']#*2
red_game['complexity']  =   red_game['complexity']# *2
# red_game

Scaled Dataset Using MinMaxScaler


In [11]:
red_game.reset_index(inplace=True)

In [12]:
red_game=red_game.dropna(axis='columns')

In [109]:
## Prep Prediciton
def weight_game(user='ecoboardgeek123', wd=rating_df):
    wd = wd.loc[wd['Username']==user]
    if (((wd['Rating'].max())-(wd['Rating'].min())) <=2):
        substractor = 5
    else:
        substractor = wd['Rating'].mean()
    wd['Rating']=wd['Rating']-substractor
    result_dict = wd.set_index('bgg_id')['Rating'].to_dict()
    # print(f"Spannweite = {wd['Rating'].max()-wd['Rating'].min()} , Substractor = {substractor}")
    return(result_dict)

test_user = weight_game()

In [100]:
rating_df['Rating'].max()-rating_df['Rating'].min()

10.0

In [59]:
### Column Dropper
def col_dropper(wd=red_game, bgg_id_list=list(test_user.keys())):
    wd_red = wd.loc[wd['bgg_id'].isin(bgg_id_list)]
    wd_red = wd_red.sum()
    wd_red = list(wd_red.loc[wd_red==0].reset_index()['index'])
    answer = wd.drop(columns=(wd_red))
    return(answer)
    

In [118]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def similar_games(games_df=red_game, alt=10, bgg_ids_with_weights={}):
    # Get the attributes of the specified games in the input list
    games_df = col_dropper(wd=games_df, bgg_id_list=list(bgg_ids_with_weights.keys()))
    selected_games_attributes = games_df[games_df['bgg_id'].isin(bgg_ids_with_weights.keys())].iloc[:, 2:]

    if selected_games_attributes.empty:
        print("No games found for the specified bgg_ids.")
        return pd.DataFrame()

    # Calculate the similarity between all games and the specified games using cosine similarity
    similarity_scores = (cosine_similarity(selected_games_attributes, games_df.iloc[:, 2:]))

    # Apply the weights to the similarity scores
    for bgg_id, weight in bgg_ids_with_weights.items():
        indices = games_df[games_df['bgg_id'] == bgg_id].index
        if len(indices) > 0:
            index = indices[0]
            similarity_scores[:, index] *= weight

    # Sum the similarity scores across the rows
    total_similarity_scores = (similarity_scores.mean(axis=0)
)
    # Add similarity scores as a new column to the DataFrame
    games_with_similarity = games_df.assign(similarity=total_similarity_scores)

    #Get rid of Games in rated Frame
    # print('bla',games_with_similarity.loc[~games_with_similarity['bgg_id'].isin(list(bgg_ids_with_weights.keys()))])#.loc[games_with_similarity['bgg_id'].isin(list[bgg_ids_with_weights.keys()])], 'test')
    games_with_similarity=games_with_similarity.loc[~games_with_similarity['bgg_id'].isin(list(bgg_ids_with_weights.keys()))]
    
    # Sort the games based on similarity scores in descending order
    top_similar_games = games_with_similarity.sort_values('similarity', ascending=False).head(alt)

    return list(top_similar_games['bgg_id'])

In [228]:
simila=similar_games(games_df=red_game, bgg_ids_with_weights=weight_game('kainolagoni'), alt=80)
simila

[40235,
 2972,
 184919,
 257731,
 2919,
 1571,
 359291,
 13285,
 161617,
 53840,
 308757,
 143520,
 204466,
 141636,
 3201,
 80771,
 18833,
 319910,
 245651,
 5811,
 38868,
 148115,
 231581,
 3482,
 135077,
 1309,
 170337,
 357890,
 200890,
 173759,
 350663,
 331017,
 38980,
 311822,
 357795,
 378873,
 59936,
 251660,
 13884,
 171579,
 217531,
 291874,
 277700,
 5231,
 311658,
 283850,
 212839,
 17071,
 234846,
 178442,
 103184,
 231207,
 45315,
 94571,
 128219,
 8233,
 233676,
 142265,
 234691,
 271150,
 280566,
 357071,
 222750,
 4195,
 156180,
 64169,
 170696,
 156303,
 191201,
 169704,
 260719,
 958,
 329024,
 159595,
 159017,
 370581,
 139627,
 40432,
 158344,
 329932]

#### Second Stage

In [229]:
# your code here
import sys

def similar_taste(wf = rating_df, include_games = simila, alt = 10, u_id= 'kainolagoni', minbo = 100):
    

    ## Filter location visited min minbo times
    filtered_df = wf[wf['Username'] == u_id]
    games = filtered_df['bgg_id'].unique()
    filtered_df = wf[wf['bgg_id'].isin(games)]
    count_table = filtered_df['Username'].value_counts().reset_index()
    count_table.columns = ['Username', 'count']
    # print(count_table)
    ##Minbo correction
    correction = 1
    while (correction== 1):
        check_table=count_table.loc[count_table['count']>=minbo]
        if len(check_table)<=5:
            minbo -=1
        elif minbo < 0:
            sys.exit("less than 5 users")
        else:
            print(f'Minbo reset to {minbo}')
            correction=0
            count_table = check_table
            

    # print(count_table)
    user_filter = wf[wf['bgg_id'].isin(include_games)]['Username'].unique()
    # print(len(user_filter))
    # count_table=count_table.loc[count_table['Username'].isin(user_filter),'Username']
    count_table=count_table.loc[(count_table['Username'].isin(user_filter)) | (count_table['Username'] == u_id),'Username']

    # print(count_table)

    keeper = wf.copy()

    wf=wf.loc[wf['Username'].isin(count_table)]

    only_known = wf.copy()
    only_known.loc[only_known['Username']==u_id,'bgg_id']
    only_known.loc[only_known['bgg_id'].isin(only_known.loc[only_known['Username']==u_id,'bgg_id'])]

    users_items = pd.pivot_table(
        data=wf.loc[wf['bgg_id'].isin(include_games)],
        values='Rating',
        index='Username',
        columns='bgg_id'
    )

    known_items = pd.pivot_table(
        data=only_known,
        values='Rating',
        index='Username',
        columns='bgg_id'
    )

    users_items.fillna(0, inplace=True)
    known_items.fillna(0, inplace=True)

    user_similarities = pd.DataFrame(
        cosine_similarity(known_items),
        columns=known_items.index,
        index=known_items.index
    )

    weights = (
        user_similarities.query("Username!=@u_id")[u_id] / sum(user_similarities.query("Username!=@u_id")[u_id])
    )
        
    # select restaurants that the inputed user has not visited
    not_visited_restaurants = users_items#.loc[users_items.index!=u_id, users_items.loc[u_id,:]==0]

    # dot product between the not-visited-restaurants and the weights
    weighted_averages = pd.DataFrame(not_visited_restaurants.T.dot(weights), columns=["predicted_rating"])

    result = weighted_averages.sort_values("predicted_rating", ascending=False).head(alt)

    result.reset_index(inplace=True)
    # result['title'] = result['bgg_id'].apply(get_title)
    # result.set_index('bgg_id', drop=True, inplace=True)
    
    return(result)

In [230]:
similar_taste()

Minbo reset to 100


,bgg_id,predicted_rating
0,45315,3.823190
1,18833,2.200591
2,231581,2.125305
3,3201,1.868859
4,277700,1.406434
5,233676,1.161681
6,13884,0.907821
7,169704,0.693681
8,80771,0.659667
9,200890,0.626821
